<p><font size="6"><b>Combining datasets Part I - concat</b></font></p>

Combining data is essential functionality in a data analysis workflow. Data is distributed in multiple files, different information needs to be merged, new data is calculated, .. and needs to be added together. Pandas provides various facilities for easily combining together Series and DataFrame objects.

In [13]:
import pandas as pd

In [14]:
# redefining the example objects

# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,area,capital,country,population
0,30510,Brussels,Belgium,11.3
1,671308,Paris,France,64.3
2,357050,Berlin,Germany,81.3
3,41526,Amsterdam,Netherlands,16.9
4,244820,London,United Kingdom,64.9


## Adding columns

As we already have seen before, adding a single column is very easy:

In [3]:
pop_density = countries['population']*1e6 / countries['area']

In [4]:
pop_density

0    370.370370
1     95.783158
2    227.699202
3    406.973944
4    265.092721
dtype: float64

In [5]:
countries['pop_density'] = pop_density

In [6]:
countries

,area,capital,country,population,pop_density
0,30510,Brussels,Belgium,11.3,370.370370
1,671308,Paris,France,64.3,95.783158
2,357050,Berlin,Germany,81.3,227.699202
3,41526,Amsterdam,Netherlands,16.9,406.973944
4,244820,London,United Kingdom,64.9,265.092721


Adding multiple columns at once is also possible:

In [10]:
countries.apply(lambda x: pd.Series([x['country'][0], x['country'][-1]]), axis=1)

,0,1
0,B,m
1,F,e
2,G,y
3,N,s
4,U,m


In [11]:
countries[['first', 'last']] = countries.apply(lambda x: pd.Series([x['country'][0], x['country'][-1]]), axis=1)

In [12]:
countries

,area,capital,country,population,pop_density,first,last
0,30510,Brussels,Belgium,11.3,370.370370,B,m
1,671308,Paris,France,64.3,95.783158,F,e
2,357050,Berlin,Germany,81.3,227.699202,G,y
3,41526,Amsterdam,Netherlands,16.9,406.973944,N,s
4,244820,London,United Kingdom,64.9,265.092721,U,m


## Concatenating data

The ``pd.concat`` function does all of the heavy lifting of combining data in different ways.

``pd.concat`` takes a list or dict of Series/DataFrame objects and concatenates them in a certain direction (`axis`) with some configurable handling of “what to do with the other axes”.


### Combining rows - ``pd.concat``

![](../img/schema-concat0.svg)

Assume we have some similar data as in `countries`, but for a set of different countries:

In [16]:
data = {'country': ['Nigeria', 'Rwanda', 'Egypt', 'Morocco', ],
        'population': [182.2, 11.3, 94.3, 34.4],
        'area': [923768, 26338 , 1010408, 710850],
        'capital': ['Abuja', 'Kigali', 'Cairo', 'Rabat']}
countries_africa = pd.DataFrame(data)
countries_africa 

,area,capital,country,population
0,923768,Abuja,Nigeria,182.2
1,26338,Kigali,Rwanda,11.3
2,1010408,Cairo,Egypt,94.3
3,710850,Rabat,Morocco,34.4


We now want to combine the rows of both datasets:

In [17]:
pd.concat([countries, countries_africa])

,area,capital,country,population
0,30510,Brussels,Belgium,11.3
1,671308,Paris,France,64.3
2,357050,Berlin,Germany,81.3
3,41526,Amsterdam,Netherlands,16.9
4,244820,London,United Kingdom,64.9
0,923768,Abuja,Nigeria,182.2
1,26338,Kigali,Rwanda,11.3
2,1010408,Cairo,Egypt,94.3
3,710850,Rabat,Morocco,34.4


If we don't want the index to be preserved:

In [5]:
pd.concat([countries, countries_africa], ignore_index=True)

,area,capital,country,population
0,30510,Brussels,Belgium,11.3
1,671308,Paris,France,64.3
2,357050,Berlin,Germany,81.3
3,41526,Amsterdam,Netherlands,16.9
4,244820,London,United Kingdom,64.9
5,923768,Abuja,Nigeria,182.2
6,26338,Kigali,Rwanda,11.3
7,1010408,Cairo,Egypt,94.3
8,710850,Rabat,Morocco,34.4


When the two dataframes don't have the same set of columns, by default missing values get introduced:

In [8]:
pd.concat([countries, countries_africa[['country', 'capital']]], ignore_index=True)

,area,capital,country,population
0,30510.0,Brussels,Belgium,11.3
1,671308.0,Paris,France,64.3
2,357050.0,Berlin,Germany,81.3
3,41526.0,Amsterdam,Netherlands,16.9
4,244820.0,London,United Kingdom,64.9
5,NaN,Abuja,Nigeria,NaN
6,NaN,Kigali,Rwanda,NaN
7,NaN,Cairo,Egypt,NaN
8,NaN,Rabat,Morocco,NaN


We can also pass a dictionary of objects instead of a list of objects. Now the keys of the dictionary are preserved as an index level:

In [18]:
pd.concat({'europe': countries, 'africa': countries_africa})

area    capital         country  population
africa 0   923768      Abuja         Nigeria       182.2
       1    26338     Kigali          Rwanda        11.3
       2  1010408      Cairo           Egypt        94.3
       3   710850      Rabat         Morocco        34.4
europe 0    30510   Brussels         Belgium        11.3
       1   671308      Paris          France        64.3
       2   357050     Berlin         Germany        81.3
       3    41526  Amsterdam     Netherlands        16.9
       4   244820     London  United Kingdom        64.9

### Combining columns  - ``pd.concat`` with ``axis=1``

![](../img/schema-concat1.svg)

Assume we have another DataFrame for the same countries, but with some additional statistics:

In [19]:
data = {'country': ['Belgium', 'France', 'Netherlands'],
        'GDP': [496477, 2650823, 820726],
        'area': [8.0, 9.9, 5.7]}
country_economics = pd.DataFrame(data).set_index('country')
country_economics

,GDP,area
country,,
Belgium,496477,8.0
France,2650823,9.9
Netherlands,820726,5.7


In [21]:
pd.concat([countries, country_economics], axis=1)

,area,capital,country,population,GDP,area
0,30510.0,Brussels,Belgium,11.3,NaN,NaN
1,671308.0,Paris,France,64.3,NaN,NaN
2,357050.0,Berlin,Germany,81.3,NaN,NaN
3,41526.0,Amsterdam,Netherlands,16.9,NaN,NaN
4,244820.0,London,United Kingdom,64.9,NaN,NaN
Belgium,NaN,NaN,NaN,NaN,496477.0,8.0
France,NaN,NaN,NaN,NaN,2650823.0,9.9
Netherlands,NaN,NaN,NaN,NaN,820726.0,5.7


`pd.concat` matches the different objects based on the index:

In [26]:
countries2 = countries.set_index('country')

In [27]:
countries2

,area,capital,population
country,,,
Belgium,30510,Brussels,11.3
France,671308,Paris,64.3
Germany,357050,Berlin,81.3
Netherlands,41526,Amsterdam,16.9
United Kingdom,244820,London,64.9


In [28]:
pd.concat([countries2, country_economics], axis=1)

,area,capital,population,GDP,area
Belgium,30510,Brussels,11.3,496477.0,8.0
France,671308,Paris,64.3,2650823.0,9.9
Germany,357050,Berlin,81.3,NaN,NaN
Netherlands,41526,Amsterdam,16.9,820726.0,5.7
United Kingdom,244820,London,64.9,NaN,NaN
